In [1]:
import pandas as pd
df = pd.read_csv("../../combined_file.csv")
df.drop(['definition', 'column_name'], axis=1, inplace=True)
df.columns

C:\Users\Franc\AppData\Local\Temp\ipykernel_1540\2358656560.py:2: DtypeWarning: Columns (0,1,2,8,9,12,13,15,17,18,19,21,22,23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../combined_file.csv")


Index(['most_recent_observation', 'most_recent_observation_type',
       'common_name', 'scientific_name', 'city', 'state',
       'longitude_coordinate', 'latitude_coordinate', 'address', 'condition',
       'height_M', 'native', 'height_binned_M',
       'diameter_breast_height_binned_CM', 'greater_metro', 'city_ID',
       'tree_ID', 'planted_date', 'retired_date', 'location_type', 'zipcode',
       'neighborhood', 'location_name', 'ward', 'district', 'overhead_utility',
       'diameter_breast_height_CM', 'percent_population'],
      dtype='object')

In [2]:
df = df[["state", "city", "common_name"]]
df = df.dropna()
df

,state,city,common_name
0,New Mexico,Albuquerque,Locust
1,New Mexico,Albuquerque,Oak
2,New Mexico,Albuquerque,Ash
3,New Mexico,Albuquerque,Scotch pine
4,New Mexico,Albuquerque,Honeylocust
...,...,...,...
5642805,District of Columbia,Washington DC,Redmond american linden
5642806,District of Columbia,Washington DC,Dogwood
5642807,District of Columbia,Washington DC,Flowering dogwood
5642808,District of Columbia,Washington DC,(see notes)


In [3]:
# Top 30 most frequent trees per state
top_trees_by_state = df['common_name'].groupby([df['state'], df['common_name']]).size().reset_index(name='count')
top_trees_by_state = top_trees_by_state.groupby('state').apply(lambda x: x.nlargest(30, 'count')).reset_index(drop=True)

# Count trees per city (only consider the trees in the top 30 of the state of the city)
result = df.merge(top_trees_by_state, on=['state', 'common_name'], how='inner')
result = result.groupby(['state', 'city', 'common_name']).size().reset_index(name='count')

print(result)

          state           city              common_name  count
0       Arizona        Phoenix            African sumac      6
1       Arizona        Phoenix              Aleppo pine      9
2       Arizona        Phoenix                  Apricot      2
3       Arizona        Phoenix  Arizona ash. velvet ash     12
4       Arizona        Phoenix                      Ash      9
...         ...            ...                      ...    ...
2322  Wisconsin  Whitefish Bay              Honeylocust     10
2323  Wisconsin  Whitefish Bay             Norway maple      2
2324  Wisconsin  Whitefish Bay                  Red oak      7
2325  Wisconsin  Whitefish Bay              Sugar maple      1
2326  Wisconsin  Whitefish Bay                White ash      3

[2327 rows x 4 columns]


In [4]:
result.to_csv("../datasets/state_species_count.csv", index=False)